Actividad 1 - Análisis Exploratorio WIDS 2024

Este notebook contiene el Análisis Exploratorio de Datos del conjunto de datos del reto WIDS Datathon 2024, enfocado en el diagnóstico temprano de cáncer de mama. El objetivo es comprender la estructura de las variables, identificar problemas de calidad (valores faltantes, inconsistencias, anomalías) y sentar las bases para la fase de preprocesamiento y modelado.

cargamos las librerías necesarias y los conjuntos de datos de entrenamiento


In [8]:
# Importación de librerías esenciales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización para mejor lectura
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

In [17]:
# Carga de los archivos
try:
    df_train = pd.read_csv(r'C:\Users\JuanCarlosCamposHerr\Desktop\MCD\3erSemestreMCD\Topicos_II\trabajofinal_topicos2\Act1 Analisis Exp WIDS\data\training.csv')
    df_test = pd.read_csv(r'C:\Users\JuanCarlosCamposHerr\Desktop\MCD\3erSemestreMCD\Topicos_II\trabajofinal_topicos2\Act1 Analisis Exp WIDS\data\test.csv')
    print("Datasets cargados exitosamente.")
except FileNotFoundError:
    print("Error: Asegúrese de que los archivos 'training.csv' y 'test.csv' estén en la ruta './data/'.")
    # Creando DataFrames vacíos para evitar errores si el archivo no existe (para demostración)
    # En un entorno real, el proceso se detendría hasta que los archivos estén disponibles.

Datasets cargados exitosamente.
